# Module 07: Playlist Management

**Difficulty**: ⭐⭐
**Estimated Time**: 30 minutes
**Prerequisites**: [Module 01: Metadata Management](01_metadata_management.ipynb), [Module 02: Audio Analysis](02_audio_analysis.ipynb)

## Learning Objectives
By the end of this notebook, you will be able to:
1. Create M3U and M3U8 playlists from your library
2. Build smart playlists using metadata criteria (genre, artist, year, quality)
3. Generate random playlists with customizable constraints
4. Import and validate existing playlists
5. Merge, split, and manage multiple playlists
6. Export playlists for use in media players

## Overview
Transform your organized library into curated playlists! This module provides powerful playlist creation and management:

- **Manual Playlists**: Create playlists from specific songs
- **Smart Playlists**: Auto-generate based on rules ("All Rock from 2000s")
- **Quality-Based**: Create playlists of high-quality or lossless files
- **Random Playlists**: Discover your library with intelligent randomization
- **Management**: Validate, merge, split, and organize playlists

### Why Playlists?
- **Curation**: Group related songs together
- **Discovery**: Rediscover forgotten music
- **Portability**: Export for car, phone, or media players
- **Organization**: Categorize by mood, activity, or occasion
- **Automation**: Let metadata create playlists for you

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import os
from pathlib import Path
from typing import List, Dict, Optional, Callable
import random
from datetime import datetime
import warnings

# Data handling
import pandas as pd
from IPython.display import display

# Audio analysis
from mutagen import File

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

print("✓ Libraries imported successfully")

In [ ]:
# Configure paths
MUSIC_LIBRARY_PATH = Path('../../music')
PLAYLIST_OUTPUT_PATH = MUSIC_LIBRARY_PATH / 'playlists'
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

# Create playlist directory if it doesn't exist
PLAYLIST_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

if not MUSIC_LIBRARY_PATH.exists():
    MUSIC_LIBRARY_PATH.mkdir(parents=True, exist_ok=True)
    print(f"Created: {MUSIC_LIBRARY_PATH.absolute()}")
else:
    print(f"✓ Music library: {MUSIC_LIBRARY_PATH.absolute()}")
    print(f"✓ Playlists folder: {PLAYLIST_OUTPUT_PATH.absolute()}")

## 2. Helper Functions (Data Collection)

In [ ]:
# Reuse helper functions from previous modules

def extract_metadata(file_path: Path) -> Dict:
    """Extract metadata from audio file."""
    metadata = {'artist': None, 'album': None, 'title': None, 'date': None, 'genre': None}
    try:
        audio = File(file_path, easy=True)
        if audio:
            for tag in metadata.keys():
                if tag in audio and audio[tag]:
                    metadata[tag] = str(audio[tag][0]) if isinstance(audio[tag], list) else str(audio[tag])
    except:
        pass
    return metadata

def get_audio_properties(file_path: Path) -> Dict:
    """Extract audio properties."""
    props = {'bitrate': None, 'duration': None}
    try:
        audio = File(file_path)
        if audio and audio.info:
            if hasattr(audio.info, 'bitrate'):
                props['bitrate'] = round(audio.info.bitrate / 1000, 0)
            if hasattr(audio.info, 'length'):
                props['duration'] = round(audio.info.length, 2)
    except:
        pass
    return props

def categorize_quality(file_path: Path, bitrate: Optional[float] = None) -> str:
    """Categorize audio quality."""
    lossless = {'.flac', '.wav', '.aiff', '.ape', '.alac'}
    if file_path.suffix.lower() in lossless:
        return 'lossless'
    if bitrate is None:
        props = get_audio_properties(file_path)
        bitrate = props.get('bitrate')
    if bitrate is None:
        return 'unknown'
    if bitrate >= 256:
        return 'high'
    elif bitrate >= 192:
        return 'medium'
    elif bitrate >= 128:
        return 'low'
    else:
        return 'poor'

def scan_library_for_playlists(library_path: Path) -> List[Dict]:
    """Scan library with metadata and audio properties for playlist creation."""
    files = []
    print(f"Scanning library: {library_path}")
    
    count = 0
    for root, dirs, filenames in os.walk(library_path):
        for filename in filenames:
            file_path = Path(root) / filename
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                count += 1
                if count % 50 == 0:
                    print(f"  Scanned {count} files...")
                
                metadata = extract_metadata(file_path)
                audio_props = get_audio_properties(file_path)
                
                files.append({
                    'filename': filename,
                    'path': str(file_path),
                    'relative_path': str(file_path.relative_to(library_path)),
                    'artist': metadata.get('artist'),
                    'album': metadata.get('album'),
                    'title': metadata.get('title'),
                    'date': metadata.get('date'),
                    'genre': metadata.get('genre'),
                    'bitrate': audio_props.get('bitrate'),
                    'duration': audio_props.get('duration'),
                    'quality_tier': categorize_quality(file_path, audio_props.get('bitrate'))
                })
    
    print(f"✓ Scan complete: {count} files")
    return files

print("✓ Helper functions loaded")

## 3. Playlist Creation Functions

In [ ]:
def create_m3u_playlist(file_list: List[Dict], 
                        playlist_name: str,
                        output_dir: Path = PLAYLIST_OUTPUT_PATH,
                        use_relative_paths: bool = True,
                        extended: bool = True) -> Path:
    """
    Create an M3U or M3U8 playlist file.
    
    Args:
        file_list: List of file dictionaries to include
        playlist_name: Name for the playlist (without extension)
        output_dir: Directory to save playlist
        use_relative_paths: Use relative paths (more portable)
        extended: Use extended M3U format with metadata
    
    Returns:
        Path to created playlist file
    """
    # Determine file extension
    extension = '.m3u8' if extended else '.m3u'
    playlist_path = output_dir / f"{playlist_name}{extension}"
    
    lines = []
    
    # Add extended M3U header if requested
    if extended:
        lines.append('#EXTM3U')
    
    # Add each file
    for file_info in file_list:
        if extended:
            # Add extended info: #EXTINF:duration,artist - title
            duration = int(file_info.get('duration', 0)) if file_info.get('duration') else -1
            artist = file_info.get('artist', 'Unknown')
            title = file_info.get('title', file_info.get('filename', 'Unknown'))
            lines.append(f'#EXTINF:{duration},{artist} - {title}')
        
        # Add file path
        if use_relative_paths and 'relative_path' in file_info:
            lines.append(file_info['relative_path'])
        else:
            lines.append(file_info['path'])
    
    # Write playlist file
    with open(playlist_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(lines))
    
    print(f"✓ Created playlist: {playlist_path}")
    print(f"  Songs: {len(file_list)}")
    total_duration = sum(f.get('duration', 0) for f in file_list if f.get('duration'))
    print(f"  Duration: {total_duration/60:.1f} minutes")
    
    return playlist_path


def format_duration(seconds: float) -> str:
    """Format duration in seconds to HH:MM:SS or MM:SS."""
    if seconds is None:
        return "Unknown"
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    if hours > 0:
        return f"{hours}:{minutes:02d}:{secs:02d}"
    else:
        return f"{minutes}:{secs:02d}"

print("✓ Playlist creation functions loaded")

## 4. Smart Playlist Functions

In [ ]:
def create_smart_playlist_by_genre(music_files: List[Dict], 
                                    genre: str,
                                    playlist_name: Optional[str] = None,
                                    limit: Optional[int] = None) -> List[Dict]:
    """
    Create playlist containing all songs of a specific genre.
    """
    matching = [f for f in music_files 
                if f.get('genre') and genre.lower() in f['genre'].lower()]
    
    if limit:
        matching = matching[:limit]
    
    if playlist_name and matching:
        create_m3u_playlist(matching, playlist_name or f"{genre}_playlist")
    
    return matching


def create_smart_playlist_by_artist(music_files: List[Dict],
                                     artist: str,
                                     playlist_name: Optional[str] = None,
                                     limit: Optional[int] = None) -> List[Dict]:
    """
    Create playlist containing all songs by a specific artist.
    """
    matching = [f for f in music_files 
                if f.get('artist') and artist.lower() in f['artist'].lower()]
    
    if limit:
        matching = matching[:limit]
    
    if playlist_name and matching:
        create_m3u_playlist(matching, playlist_name or f"{artist}_playlist")
    
    return matching


def create_smart_playlist_by_year(music_files: List[Dict],
                                   start_year: int,
                                   end_year: Optional[int] = None,
                                   playlist_name: Optional[str] = None) -> List[Dict]:
    """
    Create playlist for a specific year or year range.
    """
    if end_year is None:
        end_year = start_year
    
    matching = []
    for f in music_files:
        if f.get('date'):
            try:
                year = int(str(f['date'])[:4])  # Extract year from date
                if start_year <= year <= end_year:
                    matching.append(f)
            except (ValueError, IndexError):
                pass
    
    if playlist_name and matching:
        name = playlist_name or f"{start_year}{'_to_'+str(end_year) if end_year != start_year else ''}"
        create_m3u_playlist(matching, name)
    
    return matching


def create_smart_playlist_by_quality(music_files: List[Dict],
                                      quality_tiers: List[str],
                                      playlist_name: Optional[str] = None) -> List[Dict]:
    """
    Create playlist based on quality tier(s).
    
    Args:
        quality_tiers: List of tiers (e.g., ['lossless', 'high'])
    """
    matching = [f for f in music_files 
                if f.get('quality_tier') in quality_tiers]
    
    if playlist_name and matching:
        create_m3u_playlist(matching, playlist_name or f"{'+'.join(quality_tiers)}_quality")
    
    return matching


def create_smart_playlist_by_album(music_files: List[Dict],
                                    album: str,
                                    playlist_name: Optional[str] = None) -> List[Dict]:
    """
    Create playlist for a specific album.
    """
    matching = [f for f in music_files 
                if f.get('album') and album.lower() in f['album'].lower()]
    
    if playlist_name and matching:
        create_m3u_playlist(matching, playlist_name or f"{album}_album")
    
    return matching


def create_smart_playlist_custom(music_files: List[Dict],
                                  filter_func: Callable[[Dict], bool],
                                  playlist_name: str) -> List[Dict]:
    """
    Create playlist using a custom filter function.
    
    Args:
        filter_func: Function that returns True for files to include
    
    Example:
        # High quality rock from 2000s
        filter_func = lambda f: (
            f.get('genre') == 'Rock' and 
            f.get('quality_tier') in ['high', 'lossless'] and
            2000 <= int(f.get('date', '0')[:4]) <= 2009
        )
    """
    matching = [f for f in music_files if filter_func(f)]
    
    if matching:
        create_m3u_playlist(matching, playlist_name)
    
    return matching

print("✓ Smart playlist functions loaded")

## 5. Random Playlist Generation

In [ ]:
def create_random_playlist(music_files: List[Dict],
                          count: int,
                          playlist_name: str = "random_playlist",
                          genre: Optional[str] = None,
                          artist: Optional[str] = None,
                          quality_tier: Optional[str] = None,
                          seed: Optional[int] = None) -> List[Dict]:
    """
    Create a random playlist with optional constraints.
    
    Args:
        count: Number of songs to include
        playlist_name: Name for the playlist
        genre: Filter by genre (optional)
        artist: Filter by artist (optional)
        quality_tier: Filter by quality tier (optional)
        seed: Random seed for reproducibility (optional)
    """
    # Apply filters
    candidates = music_files.copy()
    
    if genre:
        candidates = [f for f in candidates 
                     if f.get('genre') and genre.lower() in f['genre'].lower()]
    
    if artist:
        candidates = [f for f in candidates 
                     if f.get('artist') and artist.lower() in f['artist'].lower()]
    
    if quality_tier:
        candidates = [f for f in candidates 
                     if f.get('quality_tier') == quality_tier]
    
    if not candidates:
        print("⚠️ No files match the criteria.")
        return []
    
    # Set random seed if provided
    if seed is not None:
        random.seed(seed)
    
    # Select random songs
    selected_count = min(count, len(candidates))
    selected = random.sample(candidates, selected_count)
    
    # Create playlist
    create_m3u_playlist(selected, playlist_name)
    
    return selected


def create_discovery_playlist(music_files: List[Dict],
                              count: int = 50,
                              playlist_name: str = "discovery",
                              exclude_top_artists: int = 5) -> List[Dict]:
    """
    Create a discovery playlist excluding your most-played artists.
    Helps rediscover forgotten music.
    
    Args:
        count: Number of songs
        exclude_top_artists: Number of top artists to exclude
    """
    from collections import Counter
    
    # Find top artists
    artists = [f.get('artist') for f in music_files if f.get('artist')]
    top_artists = [artist for artist, _ in Counter(artists).most_common(exclude_top_artists)]
    
    # Exclude top artists
    candidates = [f for f in music_files 
                 if f.get('artist') not in top_artists]
    
    # Select random songs
    selected_count = min(count, len(candidates))
    selected = random.sample(candidates, selected_count)
    
    create_m3u_playlist(selected, playlist_name)
    
    return selected

print("✓ Random playlist functions loaded")

## 6. Playlist Management Functions

In [ ]:
def read_m3u_playlist(playlist_path: Path, library_path: Path = MUSIC_LIBRARY_PATH) -> List[str]:
    """
    Read an M3U/M3U8 playlist and return list of file paths.
    
    Returns:
        List of file paths (absolute)
    """
    paths = []
    
    with open(playlist_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            
            # Skip comments and empty lines
            if not line or line.startswith('#'):
                continue
            
            # Convert relative paths to absolute
            if not Path(line).is_absolute():
                file_path = library_path / line
            else:
                file_path = Path(line)
            
            paths.append(str(file_path))
    
    return paths


def validate_playlist(playlist_path: Path, library_path: Path = MUSIC_LIBRARY_PATH) -> Dict:
    """
    Validate a playlist by checking if all files exist.
    
    Returns:
        Dictionary with validation results
    """
    file_paths = read_m3u_playlist(playlist_path, library_path)
    
    valid = []
    missing = []
    
    for path in file_paths:
        if Path(path).exists():
            valid.append(path)
        else:
            missing.append(path)
    
    return {
        'playlist': str(playlist_path),
        'total_entries': len(file_paths),
        'valid': len(valid),
        'missing': len(missing),
        'missing_files': missing,
        'is_valid': len(missing) == 0
    }


def merge_playlists(playlist_paths: List[Path], 
                   output_name: str,
                   remove_duplicates: bool = True) -> Path:
    """
    Merge multiple playlists into one.
    
    Args:
        playlist_paths: List of playlist files to merge
        output_name: Name for merged playlist
        remove_duplicates: Remove duplicate entries
    """
    all_paths = []
    
    for playlist_path in playlist_paths:
        paths = read_m3u_playlist(playlist_path)
        all_paths.extend(paths)
    
    if remove_duplicates:
        # Preserve order while removing duplicates
        seen = set()
        unique_paths = []
        for path in all_paths:
            if path not in seen:
                seen.add(path)
                unique_paths.append(path)
        all_paths = unique_paths
    
    # Create file info for playlist creation
    file_list = [{'path': path, 'relative_path': Path(path).name} for path in all_paths]
    
    return create_m3u_playlist(file_list, output_name, extended=False)


def list_playlists(playlist_dir: Path = PLAYLIST_OUTPUT_PATH) -> List[Dict]:
    """
    List all playlists in the playlist directory.
    """
    playlists = []
    
    for file_path in playlist_dir.glob('*.m3u*'):
        paths = read_m3u_playlist(file_path)
        playlists.append({
            'name': file_path.stem,
            'path': str(file_path),
            'format': file_path.suffix,
            'songs': len(paths),
            'modified': datetime.fromtimestamp(file_path.stat().st_mtime)
        })
    
    return playlists

print("✓ Playlist management functions loaded")

---
## 7. Usage Examples

### 7.1 Scan Library

In [ ]:
# Scan library for playlist creation
print("Scanning library...\n")
all_music_files = scan_library_for_playlists(MUSIC_LIBRARY_PATH)

print(f"\n✓ Found {len(all_music_files)} files ready for playlists")

if all_music_files:
    sample = pd.DataFrame(all_music_files[:5])
    display(sample[['filename', 'artist', 'album', 'genre', 'quality_tier']])
else:
    print("\nNo music files found. Add music to create playlists!")

### 7.2 Create Manual Playlist

In [ ]:
# Example: Create playlist from first 10 files
if all_music_files:
    sample_files = all_music_files[:10]
    
    playlist_path = create_m3u_playlist(
        sample_files,
        playlist_name="my_first_playlist",
        extended=True
    )
else:
    print("No files available for playlist.")

### 7.3 Smart Playlist by Genre

In [ ]:
# Create playlist of all Rock songs
if all_music_files:
    genre = "rock"  # Change to your genre
    
    rock_songs = create_smart_playlist_by_genre(
        all_music_files,
        genre=genre,
        playlist_name=f"{genre}_collection"
    )
    
    if rock_songs:
        print(f"\nCreated playlist with {len(rock_songs)} {genre} songs")
        df = pd.DataFrame(rock_songs[:10])
        display(df[['filename', 'artist', 'album']])
    else:
        print(f"No {genre} songs found. Try a different genre.")
else:
    print("No files available.")

### 7.4 Smart Playlist by Artist

In [ ]:
# Create playlist of all songs by specific artist
if all_music_files:
    artist = "beatles"  # Change to your artist
    
    artist_songs = create_smart_playlist_by_artist(
        all_music_files,
        artist=artist,
        playlist_name=f"{artist}_complete"
    )
    
    if artist_songs:
        print(f"\nFound {len(artist_songs)} songs by {artist}")
        df = pd.DataFrame(artist_songs)
        display(df[['filename', 'album', 'title']].head(10))
    else:
        print(f"No songs by {artist} found.")
else:
    print("No files available.")

### 7.5 Smart Playlist by Year Range

In [ ]:
# Create playlist of music from 2000s
if all_music_files:
    year_songs = create_smart_playlist_by_year(
        all_music_files,
        start_year=2000,
        end_year=2009,
        playlist_name="2000s_music"
    )
    
    if year_songs:
        print(f"\nCreated 2000s playlist with {len(year_songs)} songs")
        df = pd.DataFrame(year_songs[:10])
        display(df[['filename', 'artist', 'date']])
    else:
        print("No songs from 2000s found.")
else:
    print("No files available.")

### 7.6 Smart Playlist by Quality

In [ ]:
# Create playlist of high-quality and lossless files
if all_music_files:
    hq_songs = create_smart_playlist_by_quality(
        all_music_files,
        quality_tiers=['lossless', 'high'],
        playlist_name="high_quality_collection"
    )
    
    if hq_songs:
        print(f"\nHigh quality playlist: {len(hq_songs)} songs")
        df = pd.DataFrame(hq_songs[:10])
        display(df[['filename', 'quality_tier', 'bitrate']])
    else:
        print("No high-quality files found.")
else:
    print("No files available.")

### 7.7 Custom Smart Playlist

In [ ]:
# Example: High-quality rock from 2000s
if all_music_files:
    def my_filter(file_info):
        """Custom filter: High-quality rock from 2000s."""
        genre_match = file_info.get('genre') and 'rock' in file_info['genre'].lower()
        quality_match = file_info.get('quality_tier') in ['high', 'lossless']
        
        year_match = False
        if file_info.get('date'):
            try:
                year = int(str(file_info['date'])[:4])
                year_match = 2000 <= year <= 2009
            except:
                pass
        
        return genre_match and quality_match and year_match
    
    custom_songs = create_smart_playlist_custom(
        all_music_files,
        filter_func=my_filter,
        playlist_name="hq_2000s_rock"
    )
    
    if custom_songs:
        print(f"\nCustom playlist: {len(custom_songs)} songs")
        print("Criteria: Rock genre + High quality + 2000-2009\n")
        df = pd.DataFrame(custom_songs[:10])
        display(df[['filename', 'artist', 'date', 'quality_tier']])
    else:
        print("No songs match the criteria.")
else:
    print("No files available.")

### 7.8 Random Playlist

In [ ]:
# Create random playlist of 25 songs
if all_music_files:
    random_songs = create_random_playlist(
        all_music_files,
        count=25,
        playlist_name="random_mix_25",
        seed=42  # For reproducibility
    )
    
    if random_songs:
        print("\nRandom playlist created!")
        df = pd.DataFrame(random_songs)
        display(df[['filename', 'artist', 'genre']].head(10))
else:
    print("No files available.")

### 7.9 Discovery Playlist

In [ ]:
# Create discovery playlist (excludes top artists)
if all_music_files:
    discovery = create_discovery_playlist(
        all_music_files,
        count=30,
        playlist_name="discover_hidden_gems",
        exclude_top_artists=3
    )
    
    if discovery:
        print("\nDiscovery playlist (excluding top 3 artists):")
        df = pd.DataFrame(discovery)
        display(df[['filename', 'artist']].head(15))
else:
    print("No files available.")

### 7.10 List All Playlists

In [ ]:
# List all created playlists
playlists = list_playlists()

if playlists:
    print(f"Found {len(playlists)} playlists:\n")
    df = pd.DataFrame(playlists)
    display(df[['name', 'songs', 'format', 'modified']])
else:
    print("No playlists found. Create some playlists first!")

### 7.11 Validate Playlist

In [ ]:
# Validate a playlist (check if files exist)
if playlists:
    # Validate first playlist
    first_playlist = Path(playlists[0]['path'])
    
    validation = validate_playlist(first_playlist)
    
    print(f"Playlist Validation: {first_playlist.name}")
    print("=" * 60)
    print(f"Total entries: {validation['total_entries']}")
    print(f"Valid files: {validation['valid']}")
    print(f"Missing files: {validation['missing']}")
    print(f"Status: {'✓ Valid' if validation['is_valid'] else '⚠️ Has missing files'}")
    
    if validation['missing_files']:
        print("\nMissing files:")
        for path in validation['missing_files'][:5]:
            print(f"  - {Path(path).name}")
else:
    print("No playlists to validate.")

### 7.12 Merge Playlists

In [ ]:
# Example: Merge multiple playlists
if len(playlists) >= 2:
    # Merge first two playlists
    to_merge = [Path(playlists[0]['path']), Path(playlists[1]['path'])]
    
    merged = merge_playlists(
        to_merge,
        output_name="merged_playlist",
        remove_duplicates=True
    )
    
    print(f"\nMerged playlists:")
    print(f"  {playlists[0]['name']} ({playlists[0]['songs']} songs)")
    print(f"  {playlists[1]['name']} ({playlists[1]['songs']} songs)")
    print(f"\n→ Result: {merged.name}")
else:
    print("Need at least 2 playlists to merge.")
    print("💡 Uncomment and customize to merge specific playlists:")
    print("   merge_playlists([Path('playlist1.m3u'), Path('playlist2.m3u')], 'combined')")

## 8. Summary

### What We've Learned

In this module, we've built comprehensive playlist management capabilities:

1. **Playlist Creation**: M3U and M3U8 formats with metadata
2. **Smart Playlists**: Auto-generate by genre, artist, year, quality, album
3. **Custom Filters**: Create playlists with complex criteria
4. **Random Generation**: Discover music with intelligent randomization
5. **Discovery Mode**: Find forgotten songs by excluding top artists
6. **Management**: Validate, merge, and list playlists

### Key Functions

**Creation:**
- `create_m3u_playlist()` - Create playlist from file list
- `create_smart_playlist_by_genre()` - Genre-based playlists
- `create_smart_playlist_by_artist()` - Artist-based playlists
- `create_smart_playlist_by_year()` - Year/decade playlists
- `create_smart_playlist_by_quality()` - Quality tier playlists
- `create_smart_playlist_custom()` - Custom filter function

**Random:**
- `create_random_playlist()` - Random with optional filters
- `create_discovery_playlist()` - Rediscover forgotten music

**Management:**
- `read_m3u_playlist()` - Import existing playlists
- `validate_playlist()` - Check if files exist
- `merge_playlists()` - Combine multiple playlists
- `list_playlists()` - Show all playlists

### Playlist Formats

**M3U (Simple):**
```
path/to/song1.mp3
path/to/song2.mp3
```

**M3U8 (Extended with metadata):**
```
#EXTM3U
#EXTINF:234,Artist - Title
path/to/song1.mp3
#EXTINF:187,Another Artist - Another Title
path/to/song2.mp3
```

### Use Cases

**Genre Playlists:**
```python
create_smart_playlist_by_genre(files, "rock", "rock_collection")
create_smart_playlist_by_genre(files, "jazz", "jazz_collection")
```

**Decade Playlists:**
```python
create_smart_playlist_by_year(files, 1980, 1989, "80s_hits")
create_smart_playlist_by_year(files, 2000, 2009, "2000s_music")
```

**Quality Playlists:**
```python
# Lossless only
create_smart_playlist_by_quality(files, ['lossless'], "flac_collection")

# High quality for portable devices
create_smart_playlist_by_quality(files, ['high'], "portable_hq")
```

**Custom Combinations:**
```python
# High-quality indie from 2010s
filter_func = lambda f: (
    f.get('genre', '').lower() == 'indie' and
    f.get('quality_tier') in ['high', 'lossless'] and
    2010 <= int(f.get('date', '0')[:4]) <= 2019
)
create_smart_playlist_custom(files, filter_func, "hq_indie_2010s")
```

### Best Practices

1. **Use Extended Format**: M3U8 includes metadata for better player support
2. **Relative Paths**: More portable across different systems
3. **Validate Regularly**: Check playlists after reorganizing library
4. **Descriptive Names**: Use clear playlist names
5. **Regular Discovery**: Create discovery playlists to rediscover music
6. **Backup Playlists**: Save playlists separately from music

### Next Steps

Use playlists with:
- **Media Players**: VLC, Winamp, iTunes, foobar2000
- **Portable Devices**: Copy playlist + files to phone/car
- **Streaming**: Some apps can import M3U playlists
- **Organization**: Group similar music together

Continue to:
- **Module 08**: Quality validation for playlist files
- **Module 06**: Visualize playlist statistics
- **Custom Scripts**: Automate playlist generation

### Additional Resources

- [M3U Format Specification](https://en.wikipedia.org/wiki/M3U)
- [Media Player Compatibility](https://en.wikipedia.org/wiki/Comparison_of_media_players)
- [Playlist Best Practices](https://www.reddit.com/r/DataHoarder/wiki/playlists)